In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np 
import pandas as pd
from ortools.sat.python import cp_model


family_data_path = './input/family_data.csv'
submission_path = './input/sample_submission.csv'
pd.set_option('display.max_rows', 500)

In [14]:
def input_data():
    input_family_data = pd.read_csv(family_data_path, index_col='family_id')
    family_size = input_family_data.n_people.values.astype(np.int8)
    family_choices = np.array(input_family_data[['choice_{}'.format(i) for i in range(10)]])
    family_slots_matrix = np.zeros((5000, 100))
    for fam in range(5000):
        for choice in range(7):
            family_slots_matrix[fam, family_choices[fam, choice]-1] = choice+1
    return family_size, family_choices, family_slots_matrix

In [15]:
class ObjectivePrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution_count = 0
        self.__objective_goal = 1000000
        
    def on_solution_callback(self):
        if self.__objective_goal < self.ObjectiveValue():
            self.__set_objective_goal()
            print('Solution %i, time = %f s, objective = %i' %
                 self.__solution_count, self.WallTime(), self.ObjectiveValue())
        self.__solution_count +=1
        
    def __set_objective_goal(self):
        if self.__objective_goal > 200000:
            self.__objective_goal -= 100000
        elif self.__objective_goal > 100000:
            self.__objective_goal -= 25000
        elif self.__objective_goal > 90000:
            self.__objective_goal -= 2000
        else:
            self.__objective_goal -= 1000

In [26]:
family_size, family_choices, family_slots_matrix = input_data()

In [27]:
n_families = 5000
n_days = 100
n_slots = 9
all_families = range(n_families)
all_days = range(n_days)
#     family_size, family_choices, family_slots_matrix = input_data()

minutes_to_run = 1
assign = {}
n = {}

model = cp_model.CpModel()

for fam in all_families:
    for day in all_days:
        assign[(fam, day)] = model.NewBoolVar('x[%i][%i]' % (fam, day))

for fam in all_families:
    model.Add(sum(assign[(fam, day)] for day in all_days 
                  if family_slots_matrix[fam, day] > 0) == 1)
    model.Add(sum(assign[(fam, day)] for day in all_days 
                  if family_slots_matrix[fam, day] == 0) == 0)

for day in all_days:
    n[day] = model.NewIntVar(125, 300, 'n[%i]' % day)
    model.Add(n[day] == sum(assign[(fam, day)] * family_size[fam] for fam in all_families 
                  if family_slots_matrix[fam, day] > 0))
n[n_days] = model.NewIntVar(125, 300, 'n[%i]' % n_days)
model.Add(n[n_days] == n[n_days-1])
    

In [58]:
# 688,880,400
acc_cost = {}
diff = {}
for day in all_days:
    acc_cost[day] = model.NewIntVar(0, 5000000, 'acc_cost[%i]' % day)
    diff[day] = model.NewIntVar(0, 125, 'diff[%i]' % day)
    model.AddAbsEquality(diff[day], n[day]-n[day+1])
    model.AddDivisionEquality(acc_cost, (n[day] - 125.0), n[day] **(0.5 + diff[day] // 50.0))

model.Minimize(sum(acc_cost) + (
        sum((0 * assign[(fam, family_choices[fam][0]-1)])
        + (50 * assign[(fam, family_choices[fam][1]-1)])
        + ((50 + 9 * family_size[fam]) * assign[(fam, family_choices[fam][2]-1)])
        + ((100 + 9 * family_size[fam]) * assign[(fam, family_choices[fam][3]-1)])
        + ((200 + 9 * family_size[fam]) * assign[(fam, family_choices[fam][4]-1)])
        + ((200 + 18 * family_size[fam]) * assign[(fam, family_choices[fam][5]-1)])
        + ((300 + 18 * family_size[fam]) * assign[(fam, family_choices[fam][6]-1)])
                           for fam in all_families) * 10000)
              )


AttributeError: 'CpModel' object has no attribute 'AddAbsEquality'

In [25]:
def santa_solver():
    n_families = 5000
    n_days = 100
    n_slots = 9
    all_families = range(n_families)
    all_days = range(n_days)
#     family_size, family_choices, family_slots_matrix = input_data()

    minutes_to_run = 1
    assign = {}
    n = {}
    
    model = cp_model.CpModel()

    for fam in all_families:
        for day in all_days:
            assign[(fam, day)] = model.NewBoolVar('x[%i][%i]' % (fam, day))
            
    for fam in all_families:
        model.Add(sum(assign[(fam, day)] for day in all_days 
                      if family_slots_matrix[fam, day] > 0) == 1)
        model.Add(sum(assign[(fam, day)] for day in all_days 
                      if family_slots_matrix[fam, day] == 0) == 0)
    
    for day in all_days:
        n[day] = model.NewIntVar(125, 300, 'n[%i]' % day)
        model.Add(n[day] == sum(assign[(fam, day)] * family_size[fam] for fam in all_families 
                      if family_slots_matrix[fam, day] > 0))
    n[n_days] = model.NewIntVar(125, 300, 'n[%i]' % n_days)
    model.Add(n[n_days] == n[n_days-1])
    
#         model.Add(n[day] >= 125)
#         model.Add(n[day] <= 300)
#         model.Add(sum(assign[(fam, day)] * family_size[fam] for fam in all_families 
#                       if family_slots_matrix[fam, day] > 0) >= 125)
#         model.Add(sum(assign[(fam, day)] * family_size[fam] for fam in all_families 
#                       if family_slots_matrix[fam, day] > 0) <= 300)
    
#     pref_cost = sum((0 * x[fam][family_choices[fam][0]])
#             + (50 * x[fam][family_choices[fam][1]])
#             + ((50 + 9 * family_size[fam]) * x[fam][family_choices[fam][2]])
#             + ((100 + 9 * family_size[fam]) * x[fam][family_choices[fam][3]])
#             + ((200 + 9 * family_size[fam]) * x[fam][family_choices[fam][4]])
#             + ((200 + 18 * family_size[fam]) * x[fam][family_choices[fam][5]])
#             + ((300 + 18 * family_size[fam]) * x[fam][family_choices[fam][6]])
#                                for fam in all_families)
#     model.Add(acc_cost == sum([max(0, (n[day] - 125.0) / 400.0 * n[day] **(0.5 + n[day+1] / 50.0))]
#                               for day in range(n_days-1)))
    
    
    model.Minimize(sum((0 * assign[fam][family_choices[fam][0]-1])
            + (50 * assign[fam][family_choices[fam][1]-1])
            + ((50 + 9 * family_size[fam]) * assign[fam][family_choices[fam][2]-1])
            + ((100 + 9 * family_size[fam]) * assign[fam][family_choices[fam][3]-1])
            + ((200 + 9 * family_size[fam]) * assign[fam][family_choices[fam][4]-1])
            + ((200 + 18 * family_size[fam]) * assign[fam][family_choices[fam][5]-1])
            + ((300 + 18 * family_size[fam]) * assign[fam][family_choices[fam][6]-1])
                               for fam in all_families)\
                    + sum([max(0, (n[day] - 125.0) / 400.0 * n[day] **(0.5 + n[day+1] / 50.0))]
                              for day in range(n_days-1)))
    
    solver = cp_model.CpSolver()

    
    solver.parameters.max_time_in_seconds = minutes_to_run * 60.0
    solution_printer = ObjectivePrinter()                
    status = solver.SolveWithSolutionCallback(model, solution_printer)
    return status

santa_solver()

KeyError: 0